In [33]:
import joblib

# MODEL_NAME = 'lgbm_iso_37609_37430_10_et_v4_rand_16s_seed3333'
# MODEL_NAME = 'lgbm_iso_37105_36915_10_et_v4_rand_16s_seed4444'
# MODEL_NAME = 'lgbm_iso_37053_36902_10_et_v4_rand_16s_seed5555'
MODEL_NAME = 'catboost_iso_37267_36899_10_et_v4_rand_30s_seed3333'

models = []
for fold_id in range(10):
    model = joblib.load(f'models/{MODEL_NAME}/{fold_id}.p')
    models.append(model)

In [38]:
import lightgbm as lgb

all_feature_importances = []
all_feature_names = []

for model in models:
    try:
        feature_importances = model['gbdt_model'].booster_.feature_importance()
        all_feature_importances.append(feature_importances)

        feature_names = model['gbdt_model'].booster_.feature_name()
        all_feature_names.append(feature_names)
    except:
        feature_importances = model['gbdt_model'].get_feature_importance()
        all_feature_importances.append(feature_importances)

        feature_names = model['gbdt_model'].feature_names_
        all_feature_names.append(feature_names)

    # lgb.plot_importance(model['gbdt_model'], importance_type='gain', max_num_features=20, figsize=(10, 10))

In [39]:
import numpy as np
from scipy.stats import pearsonr

print(all_feature_names[0])
print(all_feature_importances[0])
# pearsonr(all_feature_importances[0]**0.5, all_feature_importances[1]**0.5)

['agent1_1', 'agent1_2', 'agent1_3', 'agent1_4', 'agent2_1', 'agent2_2', 'agent2_3', 'agent2_4', 'Stochastic', 'Asymmetric', 'AsymmetricForces', 'AsymmetricPiecesType', 'PlayersWithDirections', 'Cooperation', 'Team', 'Shape', 'SquareShape', 'HexShape', 'TriangleShape', 'DiamondShape', 'RectangleShape', 'SpiralShape', 'CircleShape', 'StarShape', 'SquarePyramidalShape', 'RegularShape', 'PolygonShape', 'Tiling', 'SquareTiling', 'HexTiling', 'TriangleTiling', 'SemiRegularTiling', 'MorrisTiling', 'CircleTiling', 'ConcentricTiling', 'SpiralTiling', 'AlquerqueTiling', 'MancalaBoard', 'MancalaStores', 'MancalaTwoRows', 'MancalaThreeRows', 'MancalaFourRows', 'MancalaSixRows', 'MancalaCircular', 'AlquerqueBoard', 'AlquerqueBoardWithOneTriangle', 'AlquerqueBoardWithTwoTriangles', 'AlquerqueBoardWithFourTriangles', 'AlquerqueBoardWithEightTriangles', 'ThreeMensMorrisBoard', 'ThreeMensMorrisBoardWithTwoTriangles', 'NineMensMorrisBoard', 'StarBoard', 'CrossBoard', 'KintsBoard', 'PachisiBoard', 'Fort

In [40]:
import pandas as pd
import os

output_directory_path = f'data/feature_importances/{MODEL_NAME}'
os.makedirs(output_directory_path, exist_ok=True)

for fold_id in range(10):
    feature_importances = all_feature_importances[fold_id]
    feature_names = all_feature_names[fold_id]

    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
    
    feature_importances.to_csv(f'{output_directory_path}/{fold_id}.csv', index=False)
